## Vistual inertia scheduling

vittual inertia scheduling (vis) is inherited from dcopf in opf.py

base class: dcopf <br>
vis1: dcopf + Pvsg <br>
vis2: dcopf + RoCof and fnadir + Pvsg <br>


In [38]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

from opf import dcopf

In [39]:
class vis1(dcopf):
    """
    vis0: fixed vsg up and down reserve
    """

    def __init__(self, name='dcopf', norm=None, nn_num=64):
        """
        Parameter
        ---------
        name: str
            name 
        norm: dict
            normalization dict for function fnaidir and ppeak 
            {
                Mvsg: [mean, std],
                Dvsg: [mean, std],
                Fg: [mean, std],
                Rg: [mean, std],
                Minv: [mean, std],
                Dinv: [mean, std],
            }
        nn_num: integer
            number of MLP nuerols (assume single layer MLP)
        """
        super().__init__(name)
        self.norm = norm
        self.nn_num = nn_num


    def def_type2(self, gen_idx, Mvsg, Dvsg): # used for vsg gen parameter
        """
        Define type 2 generator based on REGCV1/2

        Parameters
        ----------
        gen_idx : str
            Generator index that will be set to type 2.
        M_vsg: float
            Virtual inertia of inverter
        D_vsg: float
            Damping of inverter

        Example:
        ----------
        gen2 = ['PV_4', 'PV_5']
        M_vsg = [1, 1]
        D_vsg = [1, 1]
        ssd.def_type2(gen2, M_vsg, D_vsg)
        """
        self.gen['type'] = 1
        self.gen['Mvsg'] = 0
        self.gen['Dvsg'] = 0
        for idx, M, D in zip(gen_idx, Mvsg, Dvsg):
            row = self.gen[self.gen['idx'] == idx].index[0]
            self.gen['type'].iloc[row] = 2
            self.gen['Mvsg'].iloc[row] = M
            self.gen['Dvsg'].iloc[row] = D
    

    def build(self):
        # self.data_check()

        # --- build RTED model ---
        self.update_dict()
        self.mdl = gb.Model(self.name)
        self.mdl = self._build_vars(self.mdl)
        self.mdl = self._build_obj(self.mdl)
        self.mdl = self._build_cons(self.mdl)
        logger.info('Successfully build vis0 model.')

    def _build_vars(self, mdl):
        GEN = self.gendict.keys()

        # --- uncontrollable generators limit to p0 ---
        gencp = self.gen.copy()
        gencp['pmax'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        gencp['pmin'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        # --- offline geenrators limit to 0 ---
        gencp['pmax'][gencp.u == 0] = 0
        gencp['pmin'][gencp.u == 0] = 0


        # --- gen: pg ---
        self.pg = mdl.addVars(GEN, name='pg', vtype=gb.GRB.CONTINUOUS, obj=0,
                              ub=gencp.pmax.tolist(), lb=gencp.pmin.tolist())
        # --- RegUp, RegDn ---
        self.pru = mdl.addVars(GEN, name='pru', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        self.prd = mdl.addVars(GEN, name='prd', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        return mdl


In [40]:
## Main (test)

## Main (Test)

In [41]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_vsg.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

In [42]:
ss = vis1()

In [43]:
ss.from_andes(ssa)

In [44]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,ctrl,ramp5,ramp10,ramp30
0,PV_2,1.0,PV_2,100.0,69.0,2,0.4,0.5,0.1,1.03,1,100,200,600
1,PV_3,1.0,PV_3,100.0,69.0,3,0.4,0.5,0.1,1.01,1,100,200,600
2,PV_4,1.0,PV_4,100.0,138.0,6,0.3,1.0,0.1,1.03,1,100,200,600
3,PV_5,1.0,PV_5,100.0,69.0,8,0.3,0.5,0.1,1.03,1,100,200,600
4,PV_6,1.0,PV_6,100.0,69.0,4,-0.01,0.1,-0.1,1.01,1,100,200,600
5,PV_7,1.0,PV_7,100.0,138.0,14,0.1,0.1,-0.1,1.01,1,100,200,600
6,Slack_1,1.0,Slack_1,100.0,69.0,1,0.81442,3.0,0.5,1.03,1,100,200,600


In [45]:
gen = ['PV_6', 'PV_7']
Mvsg = [1, 1]
Dvsg = [1, 1]

ss.def_type2(gen, Mvsg, Dvsg)
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,ctrl,ramp5,ramp10,ramp30,type,Mvsg,Dvsg
0,PV_2,1.0,PV_2,100.0,69.0,2,0.4,0.5,0.1,1.03,1,100,200,600,1,0,0
1,PV_3,1.0,PV_3,100.0,69.0,3,0.4,0.5,0.1,1.01,1,100,200,600,1,0,0
2,PV_4,1.0,PV_4,100.0,138.0,6,0.3,1.0,0.1,1.03,1,100,200,600,1,0,0
3,PV_5,1.0,PV_5,100.0,69.0,8,0.3,0.5,0.1,1.03,1,100,200,600,1,0,0
4,PV_6,1.0,PV_6,100.0,69.0,4,-0.01,0.1,-0.1,1.01,1,100,200,600,2,1,1
5,PV_7,1.0,PV_7,100.0,138.0,14,0.1,0.1,-0.1,1.01,1,100,200,600,2,1,1
6,Slack_1,1.0,Slack_1,100.0,69.0,1,0.81442,3.0,0.5,1.03,1,100,200,600,1,0,0


In [46]:
ss.update_dict

<bound method system.update_dict of <__main__.vis1 object at 0x1670c96d0>>

In [47]:
ss.norm